In [1]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.p

In [2]:
test = pd.read_csv('../data/test.csv')
test

ID                                           Question
0    TEST_000  금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1    TEST_001  위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2    TEST_002  관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3    TEST_003  재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4    TEST_004  트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
..        ...                                                ...
510  TEST_510  "정보보호최고책임자"의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임...
511  TEST_511  IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512  TEST_512  하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513  TEST_513  전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...
514  TEST_514  전자서명인증업무 운영기준에 포함되지 않는 항목은 무엇인가?\n1 전자서명인증서의 유...

[515 rows x 2 columns]

In [3]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                "아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


In [6]:
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

documents = TextLoader("../data/cleaned_text_1.txt").load()
# 문서를 청크로 분할
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

# docs 변수에 분할 문서를 저장
docs = split_docs(documents)

# 1) HF 모델과 토크나이저 로드
model_id = "nlpai-lab/KULLM3"   # 예시: 원하는 모델로 교체
tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

MODEL_NAME = "jhgan/ko-sroberta-multitask"  # 또는 gte-multilingual-base, MiniLM 등

embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={
        "device": "cuda",  # 필요 시
    },
    encode_kwargs={
        "normalize_embeddings": True,
        "convert_to_tensor": True,   # ★ NumPy 대신 torch.Tensor로 받기
        "convert_to_numpy": False,   # ★ 명시적으로 끄기
    },
)

db = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="../db",   # Windows면 절대경로 권장
    collection_name="law_docs",
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
llm_model.generation_config.pad_token_id = tokenizer.pad_token_id
llm_model.generation_config.eos_token_id = tokenizer.eos_token_id

llm_model.eval()
torch.set_grad_enabled(False)
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")  # 안전한 속도 향상

# 2) 텍스트 생성 파이프라인
gen_pipe = pipeline(
    task="text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=256,
    do_sample=False,
    temperature=0.2,
)

# 3) HF 파이프라인을 LangChain Runnable로 래핑
# def hf_generate(prompt: str) -> str:
#     out = gen_pipe(prompt, return_full_text=False)
#     return out[0]["generated_text"]

def hf_generate_direct(text: str) -> str:
    # 문자열 보장(혹시 모를 PromptValue 방지)
    s = text if isinstance(text, str) else getattr(text, "to_string", lambda: str(text))()
    inputs = tokenizer(s, return_tensors="pt")
    # 모델 디바이스로 이동
    inputs = {k: v.to(llm_model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            temperature=0.2,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    # 프롬프트 이후 생성 토큰만 추출
    gen_ids = outputs[0][inputs["input_ids"].shape[1]:]
    text_out = tokenizer.decode(gen_ids, skip_special_tokens=True)
    return text_out.strip()

# llm = RunnableLambda(hf_generate)
llm = RunnableLambda(hf_generate_direct)


# 4) 벡터스토어/리트리버 (기존 db 사용 가정)
# db = Chroma(persist_directory="...")  # 이미 구성돼 있다면 재사용
# retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 20})
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})


# 5) 문서 결합 유틸
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# 6) 프롬프트 (ChatPromptTemplate 대신 순수 PromptTemplate → 문자열로 포맷)
prompt = PromptTemplate.from_template(
    "다음 컨텍스트만 사용해 간결하게 한국어로 답하세요.\n"
    "===\n{context}\n===\n질문: {input}"
)

# 7) RAG 체인 조립 (Runnable 컴포지션)
# 🔧 StringPromptValue/ChatPromptValue → str 로 변환
to_str = RunnableLambda(lambda x: x.to_string() if hasattr(x, "to_string") else str(x))

rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "input": RunnablePassthrough(),
    }
    | prompt
    | to_str      # ← 여기서 순수 문자열로 변환
    | llm
)

In [9]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    prompt = rag_chain.invoke(q)
    print("###### Question ######")
    print(q)
    print("###### Answer ######")
    print(prompt)
    pred_answer = extract_answer_only(prompt, original_question=q)
    preds.append(pred_answer)

Inference:   0%|          | 0/515 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Inference:   0%|          | 1/515 [03:44<32:01:15, 224.27s/it]

###### Question ######
금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?
1 소비자금융업
2 투자자문업
3 투자매매업
4 투자중개업
5 보험중개업
###### Answer ######
답변: 금융투자업의 구분에 해당하지 않는 것은 "소비자금융업"입니다. 소비자금융업은 금융투자업의 한 부분으로, 투자자문업, 투자매매업, 투자중개업, 보험중개업 등과 함께 금융투자업에 포함됩니다.


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Inference:   0%|          | 1/515 [04:04<34:57:26, 244.84s/it]


KeyboardInterrupt: 

In [ ]:
sample_submission = pd.read_csv('../submission/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('../submission/baseline_submission.csv', index=False, encoding='utf-8-sig')